In [3]:
from utils.processing import load_json, dump_json
from pathlib import Path
import os # pathlib барахлит
import json
import sqlite3

In [2]:
con = sqlite3.connect(r"C:\Users\egrli\FU_Projects\Курсач\intelligent_okved_embeddings\data\db\okved_20230329.sqlite3")
cur = con.cursor()

In [3]:
tables = cur.execute("""
  SELECT * FROM sqlite_master where type='table'
  -- LIMIT 5
""").fetchall()
table_names = list(
    map(
        lambda x: x[1],
        tables
    )
)

In [4]:
import pandas as pd

In [5]:
table_to_info = {}
for table_name in table_names:
    table_to_info[table_name] = pd.DataFrame(cur.execute(f"""
PRAGMA table_info({table_name});
""").fetchall()).drop(columns=[0, 3, 4, 5])

In [6]:
table_to_info['company_to_legal_member']

,1,2
0,company_inn,VARCHAR
1,member_inn,VARCHAR
2,share,FLOAT


In [76]:
table_to_info['natural_person']

,1,2
0,inn,VARCHAR
1,name,VARCHAR


In [77]:
table_to_info['supplier']

,1,2
0,id,VARCHAR
1,name,VARCHAR
2,address,VARCHAR
3,url,VARCHAR
4,inn,VARCHAR


In [78]:
table_to_info['okved']

,1,2
0,code,VARCHAR
1,name,VARCHAR
2,info,TEXT
3,section_code,VARCHAR
4,section_name,VARCHAR
5,numeric_code,VARCHAR
6,parent_numeric_code,VARCHAR


In [79]:
table_to_info['legal_entity']

,1,2
0,inn,VARCHAR
1,name,VARCHAR
2,revenue,FLOAT
3,location,VARCHAR
4,address,VARCHAR
5,region,VARCHAR
6,street,VARCHAR
7,post,VARCHAR
8,natural_representative_inn,VARCHAR
9,legal_representative_inn,VARCHAR


In [80]:
table_to_info['product'] # через url выйти на ИНН -> ОКВЭД -> ДОБАВИТЬ ЭМБЕДДИНГИ ТОВАРОВ ВАЙ НОТ

,1,2
0,id,INTEGER
1,name,VARCHAR
2,price,FLOAT
3,presence,VARCHAR
4,is_popular,BOOLEAN
5,info,TEXT
6,tags,VARCHAR
7,url,VARCHAR
8,supplier_id,VARCHAR


In [81]:
table_to_info['company_to_okved']

,1,2
0,company_inn,VARCHAR
1,okved_code,VARCHAR


In [82]:
table_to_info['company_to_natural_member']

,1,2
0,company_inn,VARCHAR
1,member_inn,VARCHAR
2,share,FLOAT


In [83]:
table_to_info['company_to_legal_member']

,1,2
0,company_inn,VARCHAR
1,member_inn,VARCHAR
2,share,FLOAT


In [84]:
table_to_info['procurement']

,1,2
0,id,VARCHAR
1,start_price,FLOAT
2,object,VARCHAR
3,method,VARCHAR
4,name,VARCHAR
5,customer_inn,VARCHAR
6,executor_inn,VARCHAR


In [87]:
# можно вытащить объем госзакупок в этмо окведе, 
# эмбеддинги услуги -> агрегнуть их по оквэд, либо оставить для компаний 
# и впихнуть как веса ребер
cur.execute(""" 
  SELECT * FROM procurement AS p
  INNER JOIN legal_entity AS le 
  ON p.executor_inn=le.inn
  LIMIT 5
""").fetchall()

[('32110356755',
  447993.0,
  'Оказание   услуг  по   организации  и  проведению   мероприятий   в пансионате  «Морозовка»',
  'СБ-АСТ: Закупка у единственного поставщика',
  'Оказание   услуг  по   организации  и  проведению   мероприятий   в пансионате  «Морозовка»',
  '5044000102',
  '7736050003',
  '7736050003',
  'ПАО "ГАЗПРОМ"',
  6000000000000.0,
  'Г.Санкт-Петербург',
  '197229, Г.Санкт-Петербург, ВН.ТЕР.Г. МУНИЦИПАЛЬНЫЙ ОКРУГ ЛАХТА-ОЛЬГИНО, ПР-КТ ЛАХТИНСКИЙ, Д. 2, К. 3, СТР. 1',
  'САНКТ-ПЕТЕРБУРГ',
  'ПР-КТ ЛАХТИНСКИЙ',
  'ПРЕДСЕДАТЕЛЬ ПРАВЛЕНИЯ',
  '782506133697',
  None,
  '46.71'),
 ('32009458795',
  1183132.8,
  'Оказание услуг газоснабжения',
  'Закупка у единственного поставщика (подрядчика, исполнителя) 223-ФЗ',
  'Оказание услуг газоснабжения',
  '7729314745',
  '7736050003',
  '7736050003',
  'ПАО "ГАЗПРОМ"',
  6000000000000.0,
  'Г.Санкт-Петербург',
  '197229, Г.Санкт-Петербург, ВН.ТЕР.Г. МУНИЦИПАЛЬНЫЙ ОКРУГ ЛАХТА-ОЛЬГИНО, ПР-КТ ЛАХТИНСКИЙ, Д. 2, К. 3, СТР. 1',
  '

In [2]:
import re
import pandas as pd
from tqdm import tqdm
import requests
import json
from bs4 import BeautifulSoup
from time import sleep
import random
okved_data = pd.read_csv('../data/okved2/okved_2014_w_sections.csv', index_col=0)
codes = okved_data['native_code'].values
codes_to_dop_codes = {k : [] for k in codes}

def find_extra_okveds(results, codes):
    res = []
    okved = r"<td>\d+\S+</td>"
    for i in range(len(results)):
        if re.search(okved, str(results[i])):
            text = results[i].text
            if text in codes:
                res.append(text)
    return res
data_root =  "C:/Users/egrli/FU_Projects/Курсач/intelligent_okved_embeddings/data/listorg/companies.json"
with open(data_root, encoding='utf-8') as fp:
    companies = json.load(fp)
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.3 Safari/605.1.15',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36',
    'Mozilla/5.0 (iPad; CPU OS 13_2_3 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.3 Mobile/15E148 Safari/604.1',
    'Mozilla/5.0 (iPhone; CPU iPhone OS 13_3 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148',
    'Mozilla/5.0 (Android 10; Mobile; rv:68.0) Gecko/68.0 Firefox/68.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:88.0) Gecko/20100101 Firefox/88.0'
]

In [4]:
for company in tqdm(companies):
    if company['okved_code'] not in codes:
        continue
    url = company['url']
    headers = {
        'User-Agent': user_agents[random.randint(0, len(user_agents)-1)]
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
    else:
        print(f"Ошибка загрузки страницы, статус код: {response.status_code}, {url}")
        break
    results = soup.find_all("td")

    codes_to_dop_codes[company['okved_code']].extend(find_extra_okveds(results, codes))
    # sleep(1)

  0%|          | 0/36435 [00:00<?, ?it/s]

In [70]:
import pickle
with open('../data/graph/okved_graph.pickle', 'rb') as fp:
        company_to_okved = pickle.load(fp)

In [71]:
company_to_okved

Graph(num_nodes=2637, num_edges=438730,
      ndata_schemes={'feat': Scheme(shape=(312,), dtype=torch.float64)}
      edata_schemes={'weight': Scheme(shape=(), dtype=torch.int64), 'type': Scheme(shape=(), dtype=torch.int64), 'train_mask': Scheme(shape=(), dtype=torch.bool), 'norm': Scheme(shape=(), dtype=torch.float32)})

In [74]:
import numpy as np
embs = np.load("../data/okved2/okved_embeddings.npy")
embs.shape

(2636, 312)

In [3]:
# %%
import pandas as pd
from pathlib import Path
import pickle
from tqdm import tqdm
import numpy as np
from itertools import combinations
import dgl
import torch as th
from typing import Union
import joblib
from copy import deepcopy
from collections import Counter

GRAPH_ROOT = Path('C:/Users/egrli/FU_Projects/Курсач/intelligent_okved_embeddings/data/graph')
OKVED_ROOT = Path('C:/Users/egrli/FU_Projects/Курсач/intelligent_okved_embeddings/data/okved2')

# %%
def fix_okved(o: str) -> str:
    """Removes extra zeros at the end of the code

    Args:
        o (str): raw OKVED code

    Returns:
        str: OKVED code with extra zeros at the end removed
    """
    if o is None:
        return

    while o[-1] == '0' and '.' in o:
        if o[-2] == '.':
            o = o[:-2]
        else:
            o = o[:-1]
    return o

def create_okveds_list(main: str, extra: list[str]) -> list[str]:
    """Create a flat list containing both main company OKVED and extra OKVEDs.

    Args:
        main (str): main company OKVED
        extra (list[str]): extra company OKVEDs

    Returns:
        list[str]: flat list containing all company OKVEDS
    """
    all_okveds = [main, *extra]
    return [fixed for o in all_okveds if (fixed := fix_okved(o)) in okved_to_idx]

def read_okved_embeddings(path: Union[Path, str], add_root_first: bool = True) -> th.Tensor:
    """Reads OKVED embeddings from the file.

    Args:
        path (Path | str): path to npy file containing OKVED embeddings
        add_root_first (bool, optional): whether to add theextra row containing
                                         zeros at the beginning. Defaults to True.

    Returns:
        th.Tensor: tensor containing OKVED embeddings
    """
    okved_embeddings = np.load(path)
    if add_root_first:
        root_emb = np.zeros((1, okved_embeddings.shape[-1]))
        okved_embeddings = np.r_[root_emb, okved_embeddings]
    okved_embeddings = th.from_numpy(okved_embeddings)
    return okved_embeddings

def push_reverse_eid_to_end(edge_df: pd.DataFrame) -> pd.DataFrame:
    """
    Сортирует ребра так, чтобы взаимообратные ребра находились на расстоянии len(edge df) // 2

    Args:
        edge_df (pd.DateFrame): таблица с ребрами

    Returns:
        pd.DataFrame: таблица c ребрами отсортированными так, чтобы взаимообратные ребра
        находились на расстоянии len(edge df) // 2
    """
    edge_df = edge_df.copy()
    edges_sort = {}
    curr_idx = 0
    edge_to_eid = {(u, v): eid for eid, (u, v) in enumerate(edge_df[['u', 'v']].values)}
    for (u, v) in edge_df[['u', 'v']].values:
        if edge_to_eid[(u, v)] not in edges_sort:
            edges_sort[edge_to_eid[(u, v)]] = curr_idx
            edges_sort[edge_to_eid[(v, u)]] = curr_idx + len(edge_df) // 2
            curr_idx += 1

    edge_df['order'] = edge_df.index.map(edges_sort)
    edge_df = edge_df.sort_values('order').reset_index(drop=True).drop(columns='order')
    return edge_df

def create_cc_edges(edata: pd.DataFrame,
                    company_to_okved_flat: pd.Series,
                    okved_to_idx: dict[str, int]) -> pd.DataFrame:
    """Creates edges based on the information about companies relationships.

    Args:
        edata (pd.DataFrame): links data from listorg preprocessed by the `build_listorg_dataset` script
        company_to_okved_flat (pd.Series): series containing all company OKVEDs row by row
        okved_to_idx (dict[str, int]): a mapping from OKVED code to its ID

    Returns:
        pd.DataFrame: The resulting dataframe contains the following columns: u_code (str), v_code (str),
                      u (int), v (int), weight (int), direction (str), type = 0.
    """
    # only company-company edges
    comp_comp_edges = edata[(edata['u'].str.startswith('company')) &
                            (edata['v'].str.startswith('company'))].copy()
    # replace url with company INN
    comp_comp_edges['u'] = comp_comp_edges['u'].map(company_inn)
    comp_comp_edges['v'] = comp_comp_edges['v'].map(company_inn)
    # there could be companies without any data collected
    comp_comp_edges.dropna(inplace=True)
    cc_edges_data = (comp_comp_edges
                    .merge(company_to_okved_flat, left_on='u', right_index=True)
                    .merge(company_to_okved_flat, left_on='v', right_index=True)
                    .loc[:, ['u', 'okved_code_x', 'v', 'okved_code_y', 'type']]
                    )
    # пока считаем все типы одинаковыми и считаем кол-во ребер между парами ОКВЭДов
    cc_edges = cc_edges_data.groupby(['okved_code_x', 'okved_code_y']).size()

    # убираем петли и редкие связи
    cc_edges = cc_edges[cc_edges.index.get_level_values(1) != cc_edges.index.get_level_values(0)]
    cc_edges = cc_edges.to_frame(name='weight').reset_index()
    cc_edges = cc_edges.query('weight > 1').reset_index(drop=True)
    # маппим названия ОКВЭД на целые числа и получаем веса ребер
    cc_edata = (cc_edges.loc[:, ['okved_code_x', 'okved_code_y', 'weight']]
                        .rename(columns={'okved_code_x': 'u_code',
                                         'okved_code_y': 'v_code'}))

    cc_edata['u'] = cc_edata['u_code'].map(okved_to_idx)
    cc_edata['v'] = cc_edata['v_code'].map(okved_to_idx)
    # Company-Company relationship type ID = 0
    cc_edata['type'] = 0
    cc_edata['direction'] = 'forward'
    # sort edges so the reverse edges are in the second half of the dataframe
    cc_edata = push_reverse_eid_to_end(cc_edata)
    half = len(cc_edata) // 2
    cc_edata.loc[half:, 'direction'] = 'reverse'
    # sanity check
    assert np.all(cc_edata[:half][['u', 'v']].values == cc_edata[half: ][['v', 'u']].values)
    return cc_edata

def create_clf_edges(okved_data: pd.DataFrame,
                     okved_to_idx: dict[str, int]) -> pd.DataFrame:
    """Creates edges based on the OKVED tree.

    Args:
        okved_data (pd.DataFrame): merged OKVED dataset generated by `build_okved_dataset.prepocess_okved_data`
        okved_to_idx (dict[str, int]): a mapping from OKVED code to its ID

    Returns:
        pd.DataFrame: The resulting dataframe contains the following columns: u_code (str), v_code (str),
                      u (int), v (int), weight (int), direction (str), type = 1.
    """
    # build okved codes tree as a dict
    digit_code_to_okved = (okved_data.loc[:, ['code_okved', 'native_code']]
                                     .set_index('code_okved')
                                     .loc[:, 'native_code']
                                     .to_dict())
    # add extra root node which is not real OKVED
    digit_code_to_okved['000000'] = 'root'
    assert len(digit_code_to_okved) - 1 == len(okved_data)

    classifier_edges = okved_data[['code_okved', 'parent_code']].copy()
    classifier_edges['code_okved'] = classifier_edges['code_okved'].map(digit_code_to_okved)
    classifier_edges['parent_code'] = classifier_edges['parent_code'].map(digit_code_to_okved)
    # add reverse edges at the end of DF s.t. the graph is not tree anymore
    classifier_edges_rev = classifier_edges.iloc[:, ::-1]
    classifier_edata = pd.DataFrame(np.concatenate([classifier_edges.values,
                                                    classifier_edges_rev.values],
                                                axis=0),
                                    columns=['u_code', 'v_code'])
    # маппим названия ОКВЭД на целые числа и получаем веса ребер из классификатора
    classifier_edata['weight'] = 1 # не очень понятный аспект
    classifier_edata['u'] = classifier_edata['u_code'].map(okved_to_idx)
    classifier_edata['v'] = classifier_edata['v_code'].map(okved_to_idx)
    # Classifier relationship type ID = 1 поправил, было 0!!!
    classifier_edata['type'] = 1
    classifier_edata['direction'] = 'forward'
    half = len(classifier_edata) // 2
    # no need to sort edges as in the other functions
    classifier_edata.loc[half:, 'direction'] = 'reverse'
    # sanity check
    assert np.all(classifier_edata[:half][['u', 'v']].values == classifier_edata[half:][['v', 'u']].values)
    return classifier_edata

def create_extra_edges(company_to_okveds_lst: dict[str, list],
                       okved_to_idx: dict[str, int]) -> pd.DataFrame:
    """Creates edges based on the information about different OKVED codes for the same companies.

    Args:
        company_to_okveds_lst (dict[str, list]): dict containing a list of possible OKVEDs for each company
        okved_to_idx (dict[str, int]): a mapping from OKVED code to its ID

    Returns:
        pd.DataFrame: The resulting dataframe contains the following columns: u_code (str), v_code (str),
                      u (int), v (int), weight (int), direction (str), type = 2.
    """
    # take each pair of codes for the same company and count the pairs' frequencies
    extra_edges = []
    for okveds in tqdm(company_to_okveds_lst.values()):
        okveds = [fix_okved(o) for o in okveds]
        # sorted is required s.t. we can be sure that for (u, v) pair from one company
        # there won't be any (v, u) pairs from another company
        extra_edges.extend(combinations(sorted(okveds), r=2))
    extra_edges_data = pd.DataFrame(extra_edges, columns=['u_code', 'v_code'])
    extra_edges = extra_edges_data.groupby(['u_code', 'v_code']).size()

    # убираем петли и редкие связи
    extra_edges = extra_edges[(extra_edges.index.get_level_values(1) !=
                               extra_edges.index.get_level_values(0))]
    extra_edges = extra_edges.to_frame(name='weight').reset_index()
    # edges between OKVEDs with the same class are too frequent so we ignore them
    not_same_class = (extra_edges['u_code'].str.slice(0, 3) !=
                      extra_edges['v_code'].str.slice(0, 3))
    # the sample is big enough to cut edges with low weights
    often_enough = extra_edges['weight'] > 10
    extra_edges = extra_edges[not_same_class & often_enough].reset_index(drop=True)
    extra_edges_rev = extra_edges.loc[:, ['v_code', 'u_code', 'weight']]
    extra_edges_rev.columns = ['u_code', 'v_code', 'weight']
    # add reverse edges to the end
    extra_edges = pd.concat([extra_edges, extra_edges_rev],
                            axis=0,
                            ignore_index=True)

    extra_edges['u'] = extra_edges['u_code'].map(okved_to_idx)
    extra_edges['v'] = extra_edges['v_code'].map(okved_to_idx)
    # Company-Company relationship type ID = 2
    extra_edges['type'] = 2
    extra_edges['direction'] = 'forward'
    # sort edges so the reverse edges are in the second half of the dataframe
    # probably unnecessary here
    extra_edges = push_reverse_eid_to_end(extra_edges)
    half = len(extra_edges) // 2
    extra_edges.loc[half:, 'direction'] = 'reverse'
    # sanity check
    assert np.all(extra_edges[:half][['u', 'v']].values == extra_edges[half:][['v', 'u']].values)
    return extra_edges

def create_gc_edges(start_prices, okved_to_idx):
    """
    Creates edges for gos contracts sum between okveds
    """
    start_prices = start_prices.rename(columns={
        "executor_okved" : "u_code",
        "customer_okved" : "v_code",
        "avg_gos" : "weight"
    }).drop(columns=[
        "count_gos", "sum_gos"
    ])
    start_prices['type'] = 3
    start_prices['direction'] = 'forward'
    start_prices = start_prices[start_prices['u_code'].apply(lambda x: x in okved_to_idx) & start_prices['v_code'].apply(lambda x: x in okved_to_idx)]
    start_prices['u'] = start_prices['u_code'].apply(lambda x: okved_to_idx[x])
    start_prices['v'] = start_prices['v_code'].apply(lambda x: okved_to_idx[x])
    start_prices = start_prices[start_prices['weight']>0]
    return start_prices[["u_code", "v_code", "weight", "u", "v", "type","direction"]]

def build_graph(okved_embeddings: th.FloatTensor,
                # gos_contracts_embeddings : th.FloatTensor,
                *edge_dfs: tuple[pd.DataFrame]) -> dgl.graph:
    """Creates DGL Graph based on edges from edge_dfs.

    Args:
        okved_embeddings (th.FloatTensor): tensor containing OKVED embeddings

    Returns:
        dgl.graph: OKVEDs graph.
    """
    # loop over directions then over different types
    # s.t. reverse edges are at the end of the table
    # after concat
    directions = ('forward', 'reverse')
    parts = [df[df['direction']==direction]
                 for direction in directions
                 for df in edge_dfs]
    all_edges = pd.concat(parts, axis=0, ignore_index=True)
    # build the graph from df
    src = all_edges['u'].values
    dst = all_edges['v'].values
    weight = th.from_numpy(all_edges['weight'].values)
    type_edges = th.from_numpy(all_edges['type'].values)

    g = dgl.graph((src, dst), num_nodes=len(idx_to_okved))
    # add extra features on nodes and edges
    g.ndata['feat'] = okved_embeddings[:2637] # HERE ADD INFO EMBS
    # g.ndata['gos'] = okved_embeddings # HERE ADD INFO EMBS
    # g.ndata['feat_2'] = gos_contracts_embeddings
    g.edata['weight'] = weight
    g.edata['type'] = type_edges

    # create train mask s.t. if the edge (u, r, v) is in train set
    # then the edge (v, r, u) will be in train set, too.
    train_mask = th.zeros(g.num_edges()).bool()
    half_all_edges = len(all_edges) // 2
    types = all_edges['type'].unique()
    g.num_edges_type = {}
    g.num_rels = len(types)
    for type_ in types:
        typed = all_edges.query('type == @type_')
        half_typed_edges = len(typed) // 2
        train_forward = (typed.iloc[:half_typed_edges]
                              .sample(frac=0.8)
                              .index)
        train_reverse = train_forward + half_all_edges
        train_mask[train_forward] = True
        train_mask[train_reverse] = True
        g.num_edges_type[type_] = len(typed)
    g.edata['train_mask'] = train_mask

    # normalize edge weights within the subgraph
    # induced by the relevant edge type
    # and add `norm` feature
    norm = dgl.nn.EdgeWeightNorm()
    g.edata['norm'] = th.zeros_like(g.edata['weight'], dtype=th.float32)
    for type_ in types:
        edges_of_type = (g.edata['type'] == type_).nonzero().flatten()
        edge_subgraph = dgl.edge_subgraph(graph=g,
                                          edges=edges_of_type,
                                          store_ids=True)
        eids = edge_subgraph.edata['_ID']
        g.edata['norm'][eids] = norm(edge_subgraph, edge_subgraph.edata['weight'].float())

    return g

# %%
if __name__ == '__main__':
    companies_ndata = pd.read_csv(GRAPH_ROOT / 'companies_ndata.csv',
                                  usecols=['url', 'INN'],
                                  dtype={'INN': str}) # информация об узлах компании- 
    ################################################### name,url,founders_amount,registration_date,okved_code,type
    company_links = pd.read_csv(GRAPH_ROOT / 'edata.csv') # ИНФО О РЕБРАХ
    okved_data = pd.read_csv(OKVED_ROOT / 'okved_2014_w_sections.csv',
                             index_col=0,
                             dtype={'code': str,
                                    'code_okved': str,
                                    'parent_code': str,
                                    'native_code': str})
    start_prices = pd.read_csv(OKVED_ROOT / "start_prices.csv", index_col=0)
    with open(OKVED_ROOT / 'company_inn_okveds_rmsp.pickle', 'rb') as fp:
        company_to_okved = pickle.load(fp) # оквэд-классификация ОКВЭДОВ (тоже граф кстати)
    okved_embeddings = read_okved_embeddings(OKVED_ROOT / 'okved_embeddings.npy') # Эмбеддинги названий ОКВЭД
    company_inn = companies_ndata.set_index('url')['INN']
    inns = set(company_inn)
    gos_contracts_embeddings = np.load(OKVED_ROOT / "okved_contact_emb.npy")
    okved_to_number_mapping = joblib.load(filename=OKVED_ROOT / 'okved_to_number_mapping.pickle')
    okved_embeddings = th.from_numpy(np.concatenate((okved_embeddings, gos_contracts_embeddings)))
    idx_in_extra_nodes = np.where(gos_contracts_embeddings[:, 0]!=0)[0]      
    extra_nodes_clear = list(okved_to_number_mapping.keys())
    extra_nodes = list(
        map(
            lambda x: x + '_',
            extra_nodes_clear
        )
    )
    idx_to_okved = {0: 'root'}
    idx_to_okved |= okved_data['native_code'].to_dict()
    # for i, okved in enumerate(extra_nodes, len(idx_to_okved)):
    #     idx_to_okved[i] = okved
    okved_to_idx = {v: k for k, v in idx_to_okved.items()}
    company_to_okveds_lst = {inn: create_okveds_list(**ok)
                                for inn, ok in tqdm(company_to_okved.items())
                                if inn in inns} # компания к ОКВЭДУ
    company_to_okved_flat = (pd.Series(company_to_okveds_lst)
                               .explode()
                               .rename('okved_code')
                               .map(fix_okved)) # ВТОРАЯ НОРМАЛЬНАЯ ФОРМА

    cc_edges = create_cc_edges(company_links, company_to_okved_flat, okved_to_idx) # ЭТО Веса ребер по числу компаний

    clf_edges = create_clf_edges(okved_data, okved_to_idx)

    extra_edges = create_extra_edges(company_to_okveds_lst, okved_to_idx) # сколько компаний имеют два кода одновременно

    # gos_contracts_edges = create_gc_edges(start_prices=start_prices, okved_to_idx=okved_to_idx)

    codes = list(set([v['main'] for v in company_to_okved.values()]))
    codes_to_extra = {code : [] for code in codes}
    for v in company_to_okved.values():
        codes_to_extra[v['main']].extend(v['extra'])
    for code in codes_to_extra:
        codes_to_extra[code] = dict(Counter(codes_to_extra[code]))
    src, tgt, weights = [], [], []

    for k, v in codes_to_extra.items():
        try:
            node = okved_to_idx[k]
            counts, nodes_extra = [], []
            for el in v:
                try:
                    nodes_extra.append(okved_to_idx[el])
                    weights.append(v[el])
                except Exception:
                    pass
            src.extend([okved_to_idx[k]] * len(nodes_extra))
            tgt.extend(nodes_extra)
        except Exception:
            pass
    src, tgt, weights = list(map(lambda x: th.tensor(x), [src, tgt, weights]))
    mask = weights >= 4
    src, tgt, weights = src[mask], tgt[mask], weights[mask]
    # weights = weights / (weights**2).sum()
    extra_okved = pd.DataFrame([src.tolist(), tgt.tolist()], index=['u', 'v']).T
    extra_okved['weight'] = weights.tolist()
    extra_okved['type'] = 4
    extra_okved['direction'] = 'forward'
    # g = build_graph(okved_embeddings, cc_edges, clf_edges, extra_edges, gos_contracts_edges)


    g = build_graph(okved_embeddings,
                    cc_edges,
                    clf_edges,
                    extra_edges,
                    # gos_contracts_edges,
                    extra_okved)
    with open(GRAPH_ROOT / 'okved_graph_new_v3.pickle', 'wb') as fp:
        pickle.dump(g, fp)


100%|██████████| 22915/22915 [00:00<00:00, 26186.19it/s]


In [4]:
g

Graph(num_nodes=2637, num_edges=894242,
      ndata_schemes={'feat': Scheme(shape=(312,), dtype=torch.float64)}
      edata_schemes={'weight': Scheme(shape=(), dtype=torch.int64), 'type': Scheme(shape=(), dtype=torch.int64), 'train_mask': Scheme(shape=(), dtype=torch.bool), 'norm': Scheme(shape=(), dtype=torch.float32)})

In [7]:
okved_embeddings.shape[0] - 2636

550

In [2]:
extra_edges

,u_code,v_code,weight,u,v,type,direction
0,01.1,02.1,22,2,129,2,forward
1,01.1,02.2,21,2,134,2,forward
2,01.1,03.1,24,2,147,2,forward
3,01.1,03.12,12,2,154,2,forward
4,01.1,03.2,27,2,159,2,forward
...,...,...,...,...,...,...,...
257585,97,96.01,25,2632,2599,2,reverse
257586,97,96.02,26,2632,2613,2,reverse
257587,97,96.03,13,2632,2614,2,reverse
257588,97,96.04,24,2632,2615,2,reverse


In [3]:
extra_okved

,u,v,weight,type,direction
0,1647,1654,3,4,forward
1,1647,1430,22,4,forward
2,1647,1537,4,4,forward
3,1647,1657,11,4,forward
4,1647,1653,4,4,forward
...,...,...,...,...,...
545543,213,1890,3,4,forward
545544,213,341,4,4,forward
545545,213,1377,4,4,forward
545546,213,1413,3,4,forward
